# Libraries and Tools

In [1]:
from utils import *
from student_utils import *
import gurobipy as grb
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Graph Class

In [2]:
"""
A class that performs data preprocessing to gather important data and to provide useful functions
"""
class Graph:
    def __init__(self, input_file):
        parsed_data = data_parser(read_file(input_file))
        self.data = {}
        self.input_file = input_file
        self.number_of_locations = parsed_data[0] 
        self.number_of_houses = parsed_data[1] 
        self.list_of_locations = parsed_data[2]  
        self.list_of_houses = parsed_data[3] 
        self.starting_location = parsed_data[4] 
        self.adjacency_matrix = parsed_data[5] 
        self.G, message = adjacency_matrix_to_graph(self.adjacency_matrix)
        if message:
            print(message)
        else:
            print("Successful creation of Graph instance")
        
    """
    Returns a matrix of shortest path distances between nodes using the Floyd-Warshall algorithm 
    If there is no path between to nodes the corresponding matrix entry will be Inf.
    """
    def get_fw_matrix(self):
        return nx.floyd_warshall_numpy(self.G)

In [3]:
G = Graph("inputs/84_50.in")

Successful creation of Graph instance


In [4]:
G.get_fw_matrix()

matrix([[ 0. , 13.9, 18.2, ..., 16.5, 15. , 14.5],
        [13.9,  0. ,  5.7, ..., 18.4, 11.5,  8.6],
        [18.2,  5.7,  0. , ..., 17.8, 10. ,  6.4],
        ...,
        [16.5, 18.4, 17.8, ...,  0. ,  8.2, 11.4],
        [15. , 11.5, 10. , ...,  8.2,  0. ,  3.6],
        [14.5,  8.6,  6.4, ..., 11.4,  3.6,  0. ]])

In [5]:
G.number_of_locations, G.number_of_houses, G.starting_location

(45, 16, '20a')

# Integer Linear Programming